In [ ]:
# Develop the linear regression algorithm from scratch
'''
Workflow:
    - Initialize parameters (w, b)
    - Compute forward_pass z = (w @ x) + b
    - Sigmoid function a = (⅟1 + np.exp(-z))
    - compute cost = (⅟n_samples) * ∑-(ylog(y_pred) + (1-y)log(1 - y_pred))
    - compute ∂/∂w = (1/n_samples) * (y - y_pred) @ X.T
    - compute ∂/∂b = (1/n_samples) * ∑(y - y_pred)
    - update parameters (w, b)
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class LogisticRegressionFromScratch:
    '''Develop the Logistic Regression Model From Scratch'''
    def __init__(self, lr: float = 0.01, epochs: int = 2000) -> None:
        self.lr = lr
        self.epochs = epochs
        self.w = None
        self.b = None
        self.cost_history = []

    def initialize_parameters(self, n_features: int) -> None:
        '''Initialize the model's parameters (w,b)'''
        self.w = np.zeros(n_features)
        self.b = 0.0

    def compute_forward_pass(self, X: np.ndarray) -> np.ndarray:
        '''Compute the initial forward pass on initialized parameters
        
        Args:
            X : feature matrix (n_samples, n_features)
        Returns:
            y_pred : predicted labels (n_samples,)
        '''
        if isinstance(X, pd.DataFrame):
            X = X.values

        y_pred = X @ self.w + self.b
        return y_pred

    def compute_cost(self, y: np.ndarray, y_pred: np.ndarray) -> int | float:
        '''Computes the cost (cross-entropy loss)
        
        Args:
            y: True Labels
            y_pred: Predicted labels

        Returns:
            Cost (scalar)
        '''
        n_samples = len(y)

        cost = (1 / n_samples) * np.sum(- (y * np.log(y_pred + 1e-5) + (1 - y) * np.log(1 - y_pred + 1e - 5)))
        return cost

    def gradient(self, X: np.ndarray, y_pred: np.ndarray, y: np.ndarray) -> tuple:
        '''Compute partial derivatives w.r.t. w and b
        
        Args:
            X : feature_matrix
            y_pred : Predicted labels
            y :True labels
            
        Returns:
            Derivatives w.r.t. w and b
        '''
        if isinstance(X,pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        n_samples = X.shape[0]

        error = y_pred - y

        dw = (1 / n_samples) * np.dot(X.T, error)
        db = (1 / n_samples) * np.sum(error)
        return dw, db

    def update_paramters(self, dw: np.ndarray, db: float) -> None:
        '''Update the parameters of w and b
        
        Args:
            dw : Derivative w.r.t. w
            db : Derivative w.r.t. b
        '''
        self.w -= self.lr * dw
        self.b -= self.lr * db